In [ ]:
import pandas as pd
import numpy as np

In [ ]:
table = pd.read_excel("SQL_data.xlsx")

**Первичный анализ имеющихся данных**

In [ ]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191759 entries, 0 to 191758
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   time           191759 non-null  datetime64[ns]
 1   tags           191759 non-null  object        
 2   question_id    191759 non-null  int64         
 3   event_type     191759 non-null  object        
 4   nav_screen     191759 non-null  object        
 5   event_subtype  12893 non-null   object        
 6   agent_role     100684 non-null  object        
 7   status         191759 non-null  int64         
 8   user_type      184401 non-null  object        
 9   reply_id       191759 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 14.6+ MB


In [ ]:
table.event_type.value_counts()

,count
event_type,
read,67780
reply,38302
agent_reply,22112
set_agent,13341
create_question,12186
ml_action,11466
closed_by_time,7358
unset_category,6370
hidden_comment,5127


In [ ]:
# read - прочитано одной из сторон
# reply - ответ пользователя
# agent_reply - ответ сотрудника
# set_agent - назначение сотрудника
# create_question - создание вопроса
# ml_action - какое-то действие, выполненное моделью
# closed_by_time - закрыто по тайм-ауту
# unset_category - снятие категории
# hidden_comment - ?
# drop_agent - снятие сотрудника (видимо для последующего назначения другого)
# closed - обращение закрыто
# set_category - назначение категории
# tag_set_manually - установка тэга
# delete_reply - удаление сообщения
# task_bind - ?
# im_csat_requested - ?
# task_closed - обращение закрыто?
# set_csat_rate - ?
# troll_closed - ?
# tag_unset_manually - снятие тэга
# change_agent_new_expired - ? (что-то со сменой сотрудника)
# change_agent_boomer_expired - ?
# extend_time - ? (время вышло?)
# change_agent_manual - ? (что-то со сменой сотрудника вручную)
# task_unbind - ?
# closed_temporary - закрыто временно? на паузе?
# hadoop_dump_sent - что-то про хадуп, встретилось всего два раза в конце обращений


# Задание 1. Тикеты, ожидающие обработки

Необходимо вывести количество тикетов, которые на данный момент ожидают отработки агентом. Для этого используйте поле status:
 - 1 — тикет отработан,
 - любое другое значение — тикет требует отработки.
Если в один и тот же момент времени записано несколько логов с разными статусами для одного тикета, строки со статусом 1 не учитываются.


In [ ]:
# Определим максимальную отметку времени для каждого тикета
table_1 = table.groupby("question_id").agg({"time" : "max"}).reset_index()
# join с общей таблицей для получения статуса в самую позднюю отметку времени
merged_table_1 = pd.merge(left=table_1, right=table, on=["question_id", "time"], how="left")
# Удалим все записи, где статус равен 1 (обработанный)
merged_table_1 = merged_table_1[merged_table_1.status != 1]

In [ ]:
# Удалим все записи, где статус равен 1 (обработанный) и посмотрим, сколько уникальных тикетов осталось необработанными
unprocessed_count = len(merged_table_1.question_id.unique())

In [ ]:
unprocessed_count

4971

In [ ]:
print(f"Необработанными на данный момент остаются {unprocessed_count} тикет.")

Необработанными на данный момент остаются 4971 тикет.


# Задание 2: Удалённые сообщения (5 баллов)

Определите, кто удаляет сообщения чаще и в каком количестве. Рассчитайте, сколько сообщений было удалено сотрудниками, а сколько — пользователями. Выберите тех, кто удаляет сообщения чаще. В ответе укажите одно число — количество удаленных сообщений выбранной группой.

In [ ]:
dict_for_task2 = dict()
user_type_list = table.user_type.value_counts().index.values # Выделим все типы user_type

In [ ]:
table[table.event_type == 'delete_reply'].user_type.value_counts() # Посмотрим тех, кто когда-либо удалял вопросы/ответы

,count
user_type,
user,600


In [ ]:
for i in user_type_list:
    try:
        dict_for_task2[i] = table[table.event_type == 'delete_reply'].user_type.value_counts().loc[i]
    except KeyError:
        dict_for_task2[i] = 0

In [ ]:
dict_for_task2

{'user': 600,
 'specagent': 0,
 'agent': 0,
 'autofaq': 0,
 'ml_agent': 0,
 'ento': 0,
 'bot': 0}

In [ ]:
max(dict_for_task2.values())

600

# Задание 3: Повторные вопросы после закрытия по причине «троллинг» (5 баллов)

Определите количество тикетов, в которых после события event_type = 'troll_closed' пользователь задал новый вопрос (reply).

In [ ]:
question_id_list_for_ex3 = table[table.event_type == "troll_closed"].question_id.unique()
table_3 = table[table.question_id.isin(question_id_list_for_ex3)].sort_values(["question_id", "time"])
# Сделал срез по тикетам, в которых было событие troll_closed

*Теперь сделаем джойн среза с самим собой, отфильтруем, чтобы события справа были всегда позже событий слева и отыщем строки, в которых слева событие troll_closed, а справа reply*

In [ ]:
table_3 = table_3[["question_id", "time", "event_type"]]
merged_table_3 = pd.merge(left=table_3, right=table_3, how="inner", on="question_id", suffixes=("_left", "_right"), sort=["question_id", "time"])

merged_table_3 = merged_table_3[(merged_table_3.event_type_left == "troll_closed")&
                                (merged_table_3.event_type_right == "reply")&
                                (merged_table_3.time_left < merged_table_3.time_right)]

In [ ]:
ex3_count = merged_table_3.shape[0]

In [ ]:
print(f"Количество тикетов, в которых было событие (event_type) troll_closed и после этого пользователь задал вопрос (reply) равно {ex3_count}")

Количество тикетов, в которых было событие (event_type) troll_closed и после этого пользователь задал вопрос (reply) равно 0


# Задание 4: Анализ тегов тикетов (10 баллов)

Посчитайте количество тикетов в разрезе тегов, где тег определяется как event_subtype LIKE 'tag_%'. Учитывайте только последний тег в каждом тикете (1 тикет — 1 тег). В ответе укажите самый популярный тег.

In [ ]:
table_4 = table.reset_index(drop=True).set_index("event_subtype")
table_4 = table_4.filter(like="tag_", axis=0).copy()
table_4 = table_4.reset_index()
# Выделяем тикеты, у которых в принципе есть назначение тега

In [ ]:
table_4.question_id.value_counts().head()
# Видим, что некоторые тикеты повторяются, это значит, что внутри них были разные назначения тегов

,count
question_id,
61896013,5
61812226,3
61798014,3
61768163,3
61707440,3


In [ ]:
table_4 = table_4.sort_values(by=['question_id', 'time'])
table_4 = table_4.drop_duplicates(subset=["question_id"], keep="last")
# Соритирую данные по тикету и времени, а затем удаляю дубликаты по номеру тикета, сохраняя только последний

Может быть такое, что тег назначили, а затем убрали и другой уже не назначили, значит по итогу такой тикет нужно убирать из рассмотрения.  
Если вдруг последним действием оказалось снятие тега (tag_unset_manually), то такой топик исключается из рассмотрения.  

In [ ]:
table_4.event_type.value_counts()

,count
event_type,
tag_set_manually,945
tag_unset_manually,1


In [ ]:
table_4 = table_4[table_4.event_type != "tag_unset_manually"]

Нашли тикет, у которого последним действием было снятие тега

Теперь наконец выведем частотность топиков в разрезе тегов

In [ ]:
table_4.groupby("event_subtype").agg({"question_id" : "count"}).sort_values(by="question_id", ascending=False)

,question_id
event_subtype,
tag_5922,233
tag_5954,53
tag_5858,53
tag_5838,50
tag_5921,41
...,...
tag_5847,1
tag_5846,1
tag_5842,1


In [ ]:
tag_str = table_4.groupby("event_subtype").agg({"question_id" : "count"}).sort_values(by="question_id", ascending=False).iloc[0].name

print(f"Самый популярный тег: {tag_str}")

Самый популярный тег: tag_5922


# Задание 5: Тег, который чаще всего ставят агенты (5 баллов)

Выведите тег (event_subtype), который чаще всего ставят агенты в тикетах. Рассматривайте только события с типом event_type = 'tag_set_manually' (значения в других столбцах могут быть любыми).

In [ ]:
table_5 = table[table.event_type == "tag_set_manually"]

In [ ]:
table_5.agent_role.unique()
# тэги могут ставить, как сотрудники, так и ml_agent-ы

array(['agent', 'ml_agent'], dtype=object)

In [ ]:
tag_str = table_5.event_subtype.value_counts().index[0]

In [ ]:
print(f"Тег, который чаще всего ставят агенты в тикетах: {tag_str}")

Тег, который чаще всего ставят агенты в тикетах: tag_5922


# Задание 6: Изменение категории тикета со статусом 0 (5 баллов)

Выведите номер тикета (question_id), в котором есть запись event_type = 'set_category', если на момент записи статус тикета (status) был равен 0.

In [ ]:
table[(table.event_type == 'set_category')&(table.status == 0)]

,time,tags,question_id,event_type,nav_screen,event_subtype,agent_role,status,user_type,reply_id
47568,2023-10-15 17:58:40,[200],61865540,set_category,22884714,NaN,agent,0,specagent,0


In [ ]:
ex6_id = table[(table.event_type == 'set_category')&(table.status == 0)].question_id.values[0]

ex6_id

61865540

# Задание 7: Расчет UWT (15)

Необходимо рассчитать показатель User Waiting Time (UWT) - среднего времени ожидания пользователем ответа сотрудника. Показатель рассчитывается на основании данных из таблицы support_questions, где фиксируются цепочки взаимодействий "вопрос пользователя" -> "ответ сотрудника". В ответе укажите среднее время в секундах с точностью до сотых.
UWT = среднее время ожидания пользователя по всем цепочкам "вопрос пользователя" -> "ответ сотрудника" за период.

Примечания:
В качестве ответа агента необходимо учитывать строки, где event_type = 'agent_reply' (с любыми значениями в других столбцах).
В качестве вопроса пользователя необходимо учитывать строки, где event_type = ‘reply’ (с любыми значениями в других столбцах).
Пользователь может написать 2 и более вопроса подряд, в этом случае учитывается вопрос, который задан раньше.
Сотрудник может дать 2 и более ответа подряд, в этом случае учитывается ответ, который дан раньше.
Удаленные вопросы и ответы исключаются из расчета.


# Задание 8: Расчёт SL (15 баллов)

Необходимо рассчитать показатель Service Level (SL) - процент ответов, данных в течение целевого времени (15 минут). Показатель рассчитывается на основании данных из таблицы support_questions, где фиксируются цепочки взаимодействий "вопрос пользователя" -> "ответ сотрудника". В ответе укажите значение с точностью до сотых.
SL = (количество цепочек с UWT ≤ 15 минут / общее количество цепочек) * 100%

Примечания:
В качестве ответа агента необходимо учитывать строки, где event_type = 'agent_reply' (с любыми значениями в других столбцах).
В качестве вопроса пользователя необходимо учитывать строки, где event_type = ‘reply’ (с любыми значениями в других столбцах).
Пользователь может написать 2 и более вопроса подряд, в этом случае учитывается вопрос, который задан раньше.
Сотрудник может дать 2 и более ответа подряд, в этом случае учитывается ответ, который дан раньше.
Удаленные вопросы и ответы исключаются из расчета.


In [ ]:
df = table.copy()

df_start = df[(df['event_type'] == 'reply') | (df['event_type'] == 'create_question')] # в отдельный массив отфильтровали события, с которых начинается расчет времени
df_start = df_start[(df_start['reply_id'] != 0) | (df_start['event_type'] == 'create_question')]
df_end = df[df['event_type'] == 'agent_reply'] # в отдельный массив отфильтровали события, на которых завершается расчет времени
df_del = df[df['event_type'] == 'delete_reply'] # в отдельный массив отфильтровали события удаления
# оставили только нужные поля
df_del = df_del[['question_id','reply_id']]

# дальше исключаем записи с удаленными сообщениями. Событий восстановления в массиве нет
df_start = pd.merge(df_start, df_del, on=['question_id', 'reply_id'], how='left', indicator=True)
df_start = df_start[df_start['_merge'] == 'left_only']
df_start = df_start.drop(columns=['_merge'])
df_end = pd.merge(df_end, df_del, on=['question_id', 'reply_id'], how='left', indicator=True)
df_end = df_end[df_end['_merge'] == 'left_only']
df_end = df_end.drop(columns=['_merge'])

# оставили только нужные поля
df_start = df_start[['question_id','time']]
df_end = df_end[['question_id','user_type','time']]

# переименовали поле для конечного массива
df_start = df_start.rename(columns={'time': 'start_time'})
df_end = df_end.rename(columns={'time': 'end_time'})
df = pd.merge(df_start, df_end, on='question_id', how='inner') #    объединили массивы по номеру тикета и оставили только совпадающие записи

df = df[df['start_time']<=df['end_time']] # отфильтровали массив, оставили только записи, где вопрос был раньше ответа
df = df.groupby(['question_id','user_type', 'start_time'])['end_time'].min().reset_index() # для каждого вопроса нашли первый ответ
df = df.groupby(['question_id','user_type', 'end_time'])['start_time'].min().reset_index() # для каждого ответа нашли первый вопрос
df = df[['question_id','user_type', 'start_time', 'end_time']] # упорядочили столбцы. В итоге получили цепочки вопрос-ответ

df['uwt'] = (df['end_time'] - df['start_time']).dt.total_seconds() # посчитали uwt - время между вопросом и ответом в новый столбец
df['results'] = 'result' # добавили столбец для удобной группировки

df['toSL_15'] = df['uwt'].apply(lambda x: 1 if x <= 900 else 0) # добавили столбец с маркером, если uwt <= 15 минут для расчета SL
df['amount'] = 1 # для простоты вычислений добавили столбец с количеством цепочек вопрос-ответ

# посчитаем для каждой недели среднее UWT, количество цепочек вопрос-ответ и количество цепочек с uwt <= 15 минут
df_78 = df.groupby('results').agg({
    'uwt': 'mean',
    'amount': 'sum',
    'toSL_15': 'sum'
}).reset_index()
df_78['sl'] = 100 * df_78[ 'toSL_15'] / df_78['amount'] # посчитали SL
df_78 = df_78[['uwt','sl']]

df_78

,uwt,sl
0,2870.578277,78.078715


# Задание 9: Определение скорости ответа сотрудников (10 баллов)

Определите, сотрудники с каким значением поля user_type дают самые быстрые ответы на вопросы пользователей. Для этого нужно рассчитать среднее (mean) время ответа для каждой группы сотрудников и определить группу с наименьшим средним временем ответа.

В качестве ответа агента необходимо учитывать строки, где event_type = 'agent_reply' (с любыми значениями в других столбцах). В качестве вопроса пользователя необходимо учитывать строки, где event_type = ‘reply’ (с любыми значениями в других столбцах).


In [ ]:
df9 = df.groupby(['user_type'])['uwt'].mean().reset_index() # посчитали средний uwt для каждого типа пользователей
df9 = df9.sort_values('uwt', ascending=True) # отсортировали результат по возростанию
df9 = df9.iloc[0] # забрали первую строку
df9[['user_type']]

,1
user_type,specagent


# Задание 10: Время до закрытия тикета (10 баллов)

Посчитайте среднее время от последнего действия в тикете до его закрытия по таймауту. Используйте только действия, которые произошли до события с типом event_type = 'closed_by_time'. Если в тикете было несколько таких событий, используйте последнее из них. В ответе дайте среднее (mean) количество дней с точностью до сотых.

Примечание: исключите из расчета действия, которые произошли после закрытия по таймауту.

In [ ]:
df = table.copy()

df_start = df[(df['event_type'] != 'closed_by_time') & (df['event_type'] != 'read')] # в отдельный массив отфильтровали события, с которых начинается расчет времени
df_start = df_start.rename(columns={'time': 'start_time'})    # переименовали столбец для удобства
df_end = df[df['event_type'] == 'closed_by_time'] # в отдельный массив отфильтровали события, на которых завершается расчет времени
df_end = df_end[['question_id','time']] # убираем лишние столбцы
df_end = df_end.rename(columns={'time': 'end_time'}) # переименовали столбец для удобства
df_end = df_end.groupby(['question_id'])['end_time'].max().reset_index() # оставляем последнее закрытие по таймауту
df = pd.merge(df_start, df_end, on='question_id', how='inner') # оставили в исходном массиве только тикеты, закрытые по таймауту
df = df[df['end_time'] >= df['start_time']] # оставили только события, которые были до закрытия по таймауту
df = df.groupby(['question_id','end_time'])['start_time'].max().reset_index() # оставляем время последнего события перед таймаутом
df['timediff'] = (df['end_time'] - df['start_time']).dt.total_seconds() # посчитали время между последним действием и закрытием по таймауту для каждого тикета в секундах
df['timediff'] = df['timediff']/60/60/24 # секунды перевели в дни

df10 = df.agg({'timediff' : 'mean'}) # посчитали среднее количество дней между последним действием и таймаутом
df10

,0
timediff,4.581036


# Задание 11: Время между назначением и сбросом агента (10 баллов)

Посчитайте среднее время от назначения сотрудника до сброса (от event_type = 'set_agent' до event_type = 'drop_agent'). В ответе укажите среднее (mean) время в секундах с точностью до сотых.

Примечания:
Учитывайте только ближайшие друг к другу события. Если было несколько событий set_agent и/или drop_agent подряд, берите последний set_agent и первый drop_agent.
Исключите из расчета сбросы, инициированные пользователями.

In [ ]:
df = table.copy()

df_start = df[(df['event_type'] != 'closed_by_time') & (df['event_type'] != 'read')] # в отдельный массив отфильтровали события, с которых начинается расчет времени
df_start = df_start.rename(columns={'time': 'start_time'})    # переименовали столбец для удобства
df_end = df[df['event_type'] == 'closed_by_time'] # в отдельный массив отфильтровали события, на которых завершается расчет времени
df_end = df_end[['question_id','time']] # убираем лишние столбцы
df_end = df_end.rename(columns={'time': 'end_time'}) # переименовали столбец для удобства
df_end = df_end.groupby(['question_id'])['end_time'].max().reset_index() # оставляем последнее закрытие по таймауту
df = pd.merge(df_start, df_end, on='question_id', how='inner') # оставили в исходном массиве только тикеты, закрытые по таймауту
df = df[df['end_time'] >= df['start_time']] # оставили только события, которые были до закрытия по таймауту

df11 = df.groupby(['question_id','end_time'])['start_time'].max().reset_index() # оставляем время последнего события перед таймаутом
df11['timediff'] = (df11['end_time'] - df11['start_time']).dt.total_seconds() # посчитали время между последним действием и закрытием по таймауту для каждого тикета в секундах
df11['timediff'] = df11['timediff']/60/60/24 # секунды перевели в дни
df11 = df11.agg({'timediff' : 'mean'}) # посчитали среднее количество дней между последним действием и таймаутом
df11

,0
timediff,4.581036
